In [283]:
import torch
torch.manual_seed(420)

In [477]:
import torch
from torch import Tensor
import csv
import pandas as pd


def read_movies():
    # read csv with movies for budget and imdb_id
    columns_of_interest = ['budget', 'imdb_id', 'revenue', 'vote_average', 'directors', 'vote_count']
    data = []
    with open('./data/movie_data_tmbd.csv', 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file, delimiter='|')
        for row in reader:
            extracted_row = {col: row[col] for col in columns_of_interest}
            data.append(extracted_row)

    movies_budget_df = pd.DataFrame(data)
    movies_budget_df = movies_budget_df.fillna({
        'budget': 0,
        'imdb_id': '',
        'title': '',
        'director': '',
        'revenue': 0,
        'vote_count': 0,
    })

    # merge movie budget with id
    link_df = pd.read_csv("./data/small/links.csv")
    link_df['imdbId'] = link_df['imdbId'].apply(lambda x: f'tt0{int(x)}')

    movies_id_df = pd.merge(movies_budget_df, link_df, left_on='imdb_id', right_on='imdbId', how='inner')
    movies_id_df['budget'] = pd.to_numeric(movies_id_df['budget'])
    movies_id_df['revenue'] = pd.to_numeric(movies_id_df['revenue'])
    movies_id_df['vote_count'] = pd.to_numeric(movies_id_df['vote_count'])
    movies_id_df = movies_id_df[movies_id_df.budget != 0]
    movies_id_df = movies_id_df[movies_id_df.revenue != 0]

    movies_info_df = pd.read_csv("./data/small/movies.csv")
    movies_df = pd.merge(movies_id_df, movies_info_df, on="movieId", how="inner")

    ratings_df = pd.read_csv("./data/small/ratings.csv")
    #ratings_df = ratings_df.iloc[:ratings_df.shape[0]//10]
    ratings_df = ratings_df[ratings_df['movieId'].isin(movies_df['movieId'])]

    return movies_df, ratings_df

movies_df, ratings_df = read_movies()
#print(movies_df.head())
print(ratings_df.head())
print(len(movies_df))
print(len(ratings_df))

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931
5       1       70     3.0  964982400
2427
55445


In [478]:
genres = movies_df['genres'].str.get_dummies('|')
movie_feat = torch.from_numpy(genres.values).to(torch.float)
movies_df['vote_average'].fillna(5.0, inplace=True)

vote_average = torch.from_numpy(movies_df['vote_average'].astype(float).values).to(torch.float).unsqueeze(-1)
revenue = torch.from_numpy(movies_df['revenue'].values).to(torch.float).unsqueeze(-1)
budget = torch.from_numpy(movies_df['budget'].values).to(torch.float).unsqueeze(-1)
vote_count = torch.from_numpy(movies_df['vote_count'].values).to(torch.float).unsqueeze(-1)

standardize = lambda x: (x - x.mean()) / x.std()

vote_average = standardize(vote_average)
revenue = standardize(revenue)
budget = standardize(budget)
vote_count = standardize(vote_count)

movie_feat = torch.cat([movie_feat, vote_count, vote_average, revenue, budget], dim=1)

print(movie_feat.shape)

torch.Size([2427, 23])


/tmp/ipykernel_2261/3551655568.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies_df['vote_average'].fillna(5.0, inplace=True)


In [479]:
ratings_df['rating_classes'], unique_classes = pd.factorize(ratings_df['rating'])

In [480]:
import pandas as pd
from ast import literal_eval

# Check if conversion is needed and perform it safely
def safe_literal_eval(s):
    try:
        if isinstance(s, str):
            return literal_eval(s)
        else:
            return s  # assuming s is already the correct format
    except (ValueError, SyntaxError):
        return None

movies_df['directors'] = movies_df['directors'].apply(safe_literal_eval)

# Check the result after conversion
print(movies_df['directors'].head())

0    [{'credit_id': '52fe4640c3a36847f80f4a8b', 'de...
1    [{'credit_id': '52fe4652c3a36847f80f8afb', 'de...
2    [{'credit_id': '52fe466bc3a36847f80fdfb9', 'de...
3    [{'credit_id': '52fe466dc3a36847f80fe76d', 'de...
4    [{'credit_id': '57aa51cac3a368570a000786', 'de...
Name: directors, dtype: object


In [481]:
# Explode the DataFrame on the 'directors' column if the data is correctly formatted as lists of dictionaries
directors_expanded = movies_df.explode('directors').dropna()

# Check the first few rows to ensure explosion worked
print(directors_expanded['directors'].head())

0    {'credit_id': '52fe4640c3a36847f80f4a8b', 'dep...
1    {'credit_id': '52fe4652c3a36847f80f8afb', 'dep...
2    {'credit_id': '52fe466bc3a36847f80fdfb9', 'dep...
2    {'credit_id': '52fe466bc3a36847f80fdfb3', 'dep...
3    {'credit_id': '52fe466dc3a36847f80fe76d', 'dep...
Name: directors, dtype: object


In [482]:
# Normalize the directors' dictionary data into separate DataFrame columns
if not directors_expanded['directors'].isnull().all():  # Check if all values are not None
    directors_expanded[['director_id', 'director_name']] = directors_expanded['directors'].apply(lambda d: pd.Series({'id': d['id'], 'name': d['name']}) if pd.notna(d) else pd.Series({'id': None, 'name': None}))
else:
    print("No valid director data available.")

# Check if the new columns are filled correctly
print(directors_expanded[['director_id', 'director_name']].head())

   director_id  director_name
0         9181   Edward Zwick
1        59291  Alexandre Aja
2        53068     Ryan Fleck
2        53069     Anna Boden
3         3026     Rob Reiner


In [483]:
import pandas as pd
from ast import literal_eval

# Convert the 'vote_average' to float
directors_expanded['vote_average'] = pd.to_numeric(directors_expanded['vote_average'], errors='coerce')

# Check if the conversion was successful
print(directors_expanded[['director_name', 'vote_average']].head())

   director_name  vote_average
0   Edward Zwick           7.0
1  Alexandre Aja           5.3
2     Ryan Fleck           7.0
2     Anna Boden           7.0
3     Rob Reiner           7.7


In [484]:
# Group by director name and calculate mean ratings and count of movies
director_stats = directors_expanded.groupby('director_name').agg({
    'vote_average': ['mean', 'count']
}).reset_index()

# Flatten the multi-level column headers
director_stats.columns = ['director', 'average_rating', 'movie_counts']
director_stats = director_stats[['average_rating','movie_counts']]
# Display the resulting DataFrame
print(director_stats.shape)

(1229, 2)


In [485]:
director_stats['average_rating'] = standardize(director_stats['average_rating'])
director_stats['movie_counts'] = standardize(director_stats['movie_counts'])

In [486]:
director_stats = torch.from_numpy(director_stats.values).to(torch.float)
director_stats.shape

torch.Size([1229, 2])

In [487]:
directors_expanded = directors_expanded[['movieId', 'director_name', 'director_id']]
movies_extended_df = pd.merge(movies_df, directors_expanded, on='movieId', how='left')
movies_extended_df = movies_extended_df.dropna(subset=['director_name'])
print(len(movies_extended_df['director_name'].unique()))

1229


In [488]:
print(movies_extended_df['director_name'].unique())

['Edward Zwick' 'Alexandre Aja' 'Ryan Fleck' ... 'Oliver Parker'
 'Adam Elliot' 'Lance Mungia']


In [489]:
movies_extended_df['movieId'].unique()

array([82167, 79879, 80693, ...,  2275,  4141,  6289])

In [400]:
unique_director_id = movies_extended_df['director_id'].unique()
unique_director_id = pd.DataFrame(data={
    'director_id': unique_director_id,
    'mappedId': pd.RangeIndex(len(unique_director_id))
})
unique_movie_id = movies_extended_df['movieId'].unique()
unique_movie_id = pd.DataFrame(data={
    'movieId': unique_movie_id,
    'mappedId': pd.RangeIndex(len(unique_movie_id))
})

director_director_id = pd.merge(movies_extended_df['director_id'], unique_director_id, on='director_id', how='left')
director_director_id = torch.from_numpy(director_director_id['mappedId'].values)
director_movie_id = pd.merge(movies_extended_df['movieId'], unique_movie_id, on='movieId', how='left')
director_movie_id = torch.from_numpy(director_movie_id['mappedId'].values)

edge_index_director_to_movie = torch.stack([director_director_id, director_movie_id], dim=0)
print(edge_index_director_to_movie.shape)

torch.Size([2, 2605])


In [490]:
# construct a compact representation of the data
unique_user_id = ratings_df['userId'].unique()
unique_user_id = pd.DataFrame(data={
    'userId': unique_user_id,
    'mappedId': pd.RangeIndex(len(unique_user_id)),
})
unique_movie_id = ratings_df['movieId'].unique()
unique_movie_id = pd.DataFrame(data={
    'movieId': unique_movie_id,
    'mappedId': pd.RangeIndex(len(unique_movie_id)),
})

ratings_user_id = pd.merge(ratings_df['userId'], unique_user_id, on='userId', how='left')
ratings_user_id = torch.from_numpy(ratings_user_id['mappedId'].values)
ratings_movie_id = pd.merge(ratings_df['movieId'], unique_movie_id, on='movieId', how='left')
ratings_movie_id = torch.from_numpy(ratings_movie_id['mappedId'].values)

edge_index_user_to_movie = torch.stack([ratings_user_id, ratings_movie_id], dim=0)

In [402]:
print(len(movies_extended_df['director_id'].unique()))
print(director_stats.shape)

1229
torch.Size([1229, 2])


In [403]:
torch.from_numpy(ratings_df['rating'].values).shape

torch.Size([55445])

In [404]:
edge_index_user_to_movie.shape

torch.Size([2, 55445])

In [405]:
print(len(ratings_df['movieId'].unique()))

2427


In [498]:
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

data = HeteroData()
# save node indices
data['user'].node_id = torch.arange(len(unique_user_id))
data['movie'].node_id = torch.arange(len(unique_movie_id))
data['director'].node_id = torch.arange(len(unique_director_id))

# add node features
data['movie'].x = movie_feat
data['director'].x = director_stats
data['user', 'rates', 'movie'].edge_index = edge_index_user_to_movie
data['user', 'rates', 'movie'].edge_label = torch.from_numpy(ratings_df['rating'].values).to(torch.long) # TODO: this cuts off the .5 steps
mask = data['user', 'rates', 'movie'].edge_label >= 4
data['user', 'rates', 'movie'].edge_index = data['user', 'rates', 'movie'].edge_index[:, mask]
#data['user', 'rates', 'movie'].edge_label = torch.ones(data['user', 'rates', 'movie'].edge_index.shape[1])
del data['user', 'rates', 'movie'].edge_label 
data['director', 'directs', 'movie'].edge_index = edge_index_director_to_movie
#data['director', 'directs', 'movie'].edge_label = torch.ones(edge_index_director_to_movie.shape[1]).to(torch.long)
del data['director', 'directs', 'movie'].edge_label
data = T.ToUndirected()(data)
print(data)

HeteroData(
  user={ node_id=[609] },
  movie={
    node_id=[2427],
    x=[2427, 23],
  },
  director={
    node_id=[1229],
    x=[1229, 2],
  },
  (user, rates, movie)={ edge_index=[2, 25903] },
  (director, directs, movie)={ edge_index=[2, 2605] },
  (movie, rev_rates, user)={ edge_index=[2, 25903] },
  (movie, rev_directs, director)={ edge_index=[2, 2605] }
)


In [407]:
print(edge_index_user_to_movie.shape)
print(data['user', 'rates', 'movie'].edge_index.shape)
print(data['user', 'rates', 'movie'].edge_label.shape)

torch.Size([2, 55445])
torch.Size([2, 25903])


AttributeError: 'EdgeStorage' object has no attribute 'edge_label'

In [ ]:
print(len(unique_director_id))
print(director_stats.shape)
print(movie_feat.shape)
print(len(data['movie'].node_id))

1229
torch.Size([1229, 2])
torch.Size([2427, 23])
2427


In [ ]:
print(data['user', 'rates', 'movie'].edge_label)
print(ratings_df['rating'].values)

AttributeError: 'EdgeStorage' object has no attribute 'edge_label'

In [499]:
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    disjoint_train_ratio=0.3,
    neg_sampling_ratio=2.0,
    add_negative_train_samples=False,
    edge_types=[("user", "rates", "movie"), ("director", "directs", "movie")],
    rev_edge_types=[("movie", "rev_rates", "user"), ("movie", "rev_directs", "director")],
)
train_data, val_data, test_data = transform(data)

In [409]:
print(data['user', 'rates', 'movie'].edge_label.unique())
print(torch.from_numpy(ratings_df['rating'].values).to(torch.long).unique())
train_data['user', 'rates', 'movie'].edge_label.unique()

AttributeError: 'EdgeStorage' object has no attribute 'edge_label'

In [500]:
from torch_geometric.loader import LinkNeighborLoader
# Example configuration with correct tuple formatting for edge_label_index:
train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[20,10, 10],
    edge_label_index=(('user', 'rates', 'movie'), train_data['user', 'rates', 'movie'].edge_label_index),  # Correct tuple format
    # edge_label=train_data['user', 'rates', 'movie'].edge_label,  # Labels for the edges
    batch_size=128,
    shuffle=True,
    neg_sampling_ratio=2.0  # Example negative sampling ratio
)

val_sample = next(iter(train_loader))
print(val_sample)


HeteroData(
  user={
    node_id=[595],
    n_id=[595],
    num_sampled_nodes=[4],
  },
  movie={
    node_id=[1595],
    x=[1595, 23],
    n_id=[1595],
    num_sampled_nodes=[4],
  },
  director={
    node_id=[576],
    x=[576, 2],
    n_id=[576],
    num_sampled_nodes=[4],
  },
  (user, rates, movie)={
    edge_index=[2, 8269],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[8269],
    num_sampled_edges=[3],
    input_id=[128],
  },
  (director, directs, movie)={
    edge_index=[2, 929],
    edge_label=[625],
    edge_label_index=[2, 625],
    e_id=[929],
    num_sampled_edges=[3],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 6525],
    e_id=[6525],
    num_sampled_edges=[3],
  },
  (movie, rev_directs, director)={
    edge_index=[2, 1052],
    e_id=[1052],
    num_sampled_edges=[3],
  }
)


In [501]:
# Define the validation seed edges:
edge_label_index = val_data["user", "rates", "movie"].edge_label_index
edge_label = val_data["user", "rates", "movie"].edge_label
val_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=[20, 10],
    edge_label_index=(("user", "rates", "movie"), edge_label_index),
    # edge_label=edge_label,
    batch_size=128*3,
    shuffle=False,
    neg_sampling_ratio=2.0,
)
val_sample = next(iter(val_loader))   

In [502]:
val_sample

HeteroData(
  user={
    node_id=[604],
    n_id=[604],
    num_sampled_nodes=[4],
  },
  movie={
    node_id=[2009],
    x=[2009, 23],
    n_id=[2009],
    num_sampled_nodes=[4],
  },
  director={
    node_id=[859],
    x=[859, 2],
    n_id=[859],
    num_sampled_nodes=[4],
  },
  (user, rates, movie)={
    edge_index=[2, 11369],
    edge_label=[1152],
    edge_label_index=[2, 1152],
    e_id=[11369],
    num_sampled_edges=[3],
    input_id=[384],
  },
  (director, directs, movie)={
    edge_index=[2, 1720],
    edge_label=[780],
    edge_label_index=[2, 780],
    e_id=[1720],
    num_sampled_edges=[3],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 8673],
    e_id=[8673],
    num_sampled_edges=[3],
  },
  (movie, rev_directs, director)={
    edge_index=[2, 1802],
    e_id=[1802],
    num_sampled_edges=[3],
  }
)

In [503]:
from torch_geometric.nn import GraphConv, to_hetero, SAGEConv, GraphSAGE

class GNN(torch.nn.Module):
    def __init__(self, num_features, hidden_dim, layers):
        super().__init__()
        input_dim = num_features
        self.conv = torch.nn.ModuleList()
        for _ in range(layers):
            self.conv.append(SAGEConv(input_dim, hidden_dim))
            input_dim = hidden_dim
    
    def forward(self, x, edge_index):#, edge_weight):
        for layer in self.conv:
            x = layer(x, edge_index)
            x = torch.relu(x)
        return x

class Classifier(torch.nn.Module):
    def __init__(self, hidden_dim, num_classes):
        super().__init__()
        self.fc1 = torch.nn.Linear(2*hidden_dim, hidden_dim)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_dim, num_classes)  # Final layer that outputs class scores

    def forward(self, x_user: Tensor, x_movie: Tensor, edge_label_index: Tensor) -> Tensor:
        # Extract features for users and movies according to the edge indices
        edge_feat_user = x_user[edge_label_index[0]]
        edge_feat_movie = x_movie[edge_label_index[1]]

        # Concatenate user and movie features
        x = torch.cat((edge_feat_user, edge_feat_movie), dim=1)  # Concatenation along the feature dimension

        # Pass the concatenated vector through the dense layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


class Model(torch.nn.Module):
    # TODO: add data in function header?
    def __init__(self, num_features, hidden_dim, num_classes, layers):
        super().__init__()
        self.movie_lin = torch.nn.Linear(23, hidden_dim) # TODO: thats the number of neighbors?
        self.director_lin = torch.nn.Linear(2, hidden_dim)
        self.user_emb = torch.nn.Embedding(data['user'].num_nodes, hidden_dim)
        self.movie_emb = torch.nn.Embedding(data['movie'].num_nodes, hidden_dim)
        self.director_emb = torch.nn.Embedding(data['director'].num_nodes, hidden_dim)
        self.gnn = GNN(num_features, hidden_dim, layers)
        self.gnn = to_hetero(self.gnn, metadata=data.metadata(), aggr='sum')
        self.classifier = Classifier(hidden_dim, num_classes)

    def forward(self, data:HeteroData) -> Tensor:
        x_dict = {
            "user": self.user_emb(data['user'].node_id),
            "movie": self.movie_lin(data['movie'].x) + self.movie_emb(data['movie'].node_id),
            "director": self.director_lin(data['director'].x) + self.director_emb(data['director'].node_id) 
        }
        x_dict = self.gnn(x_dict, data.edge_index_dict)#, data.edge_weight_dict)
        pred = self.classifier(
            x_dict["user"],
            x_dict["movie"],
            data["user", "rates", "movie"].edge_label_index,
        )
        return pred

model = Model(32, 64, 1, 3)
print(model)

Model(
  (movie_lin): Linear(in_features=23, out_features=32, bias=True)
  (director_lin): Linear(in_features=2, out_features=32, bias=True)
  (user_emb): Embedding(609, 32)
  (movie_emb): Embedding(2427, 32)
  (director_emb): Embedding(1229, 32)
  (gnn): GraphModule(
    (conv): ModuleList(
      (0-2): 3 x ModuleDict(
        (user__rates__movie): SAGEConv(32, 32, aggr=mean)
        (director__directs__movie): SAGEConv(32, 32, aggr=mean)
        (movie__rev_rates__user): SAGEConv(32, 32, aggr=mean)
        (movie__rev_directs__director): SAGEConv(32, 32, aggr=mean)
      )
    )
  )
  (classifier): Classifier(
    (fc1): Linear(in_features=64, out_features=32, bias=True)
    (relu): ReLU()
    (fc2): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [464]:
from torch_geometric.nn import GraphConv, HeteroConv, to_hetero
from torch.nn.functional import binary_cross_entropy_with_logits

num_features = 64
hidden_dim = 64

model = Model(64, 64, 1, 8)
sample = next(iter(train_loader))
output = model(sample)
target = sample['user', 'rates', 'movie'].edge_label
print(output.squeeze(-1)[0])
print(target[0])
loss = binary_cross_entropy_with_logits(output.squeeze(-1), target)
print(loss)

tensor(-0.1324, grad_fn=<SelectBackward0>)
tensor(1.)
tensor(0.6731, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


In [497]:
import tqdm
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

#model = torch.load("./model/auc_8092.pt")
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
save = False


for epoch in range(15):
    total_loss = 0
    total_examples = 0
    model.train()
    for train_sample in tqdm.tqdm(train_loader):
        optimizer.zero_grad()
        train_sample.to(device)
        pred = model(train_sample)
        label = train_sample['user', 'rates', 'movie'].edge_label
        loss = F.binary_cross_entropy_with_logits(pred, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * label.size(0) # TODO: numel?!
        total_examples += label.size(0)

    
    # Validation phase
    model.eval()
    preds = []
    ground_truths = []
    for sampled_data in tqdm.tqdm(val_loader):
        with torch.no_grad():
            sampled_data.to(device)
            preds.append(model(sampled_data))
            ground_truths.append(sampled_data["user", "rates", "movie"].edge_label)
    pred = torch.sigmoid(pred)
    pred = torch.cat(preds, dim=0).cpu().numpy()
    ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
    auc = roc_auc_score(ground_truth, pred)
    print(f'Epoch {epoch + 1:03d}, Loss: {total_loss / len(train_loader.dataset):.4f}, Validation AUC: {auc:.4f}')
    

if save: torch.save(model, f"./model/auc_{int(auc*10000)}.pt")

Device: cuda


100%|██████████████████████████████████████████| 21/21 [00:00<00:00, 392.07it/s]


Epoch 001, Loss: 1.8949, Validation AUC: 0.5871


100%|██████████████████████████████████████████| 21/21 [00:00<00:00, 239.08it/s]


Epoch 002, Loss: 1.4756, Validation AUC: 0.6072


...

Epoch 013, Loss: 0.9453, Validation AUC: 0.6221


100%|██████████████████████████████████████████| 21/21 [00:00<00:00, 427.63it/s]


Epoch 014, Loss: 0.9738, Validation AUC: 0.6260


100%|██████████████████████████████████████████| 21/21 [00:00<00:00, 421.98it/s]

Epoch 015, Loss: 0.9399, Validation AUC: 0.6260
